https://wikidocs.net/253836

In [7]:
from dotenv import load_dotenv
import nest_asyncio

load_dotenv()
nest_asyncio.apply()

In [8]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# 파서 설정
parser = LlamaParse(
    result_type="markdown",  # "markdown"과 "text" 사용 가능
    num_workers=8,  # worker 수 (기본값: 4)
    verbose=True,
    language="ko",
    
)

file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(
    input_files = ["../assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf"]
    , file_extractor = file_extractor
).load_data()
documents

Started parsing the file under job_id a4cd518d-3538-44bc-b0e7-2200184ee55f
.............

[Document(id_='67967414-bb12-4efe-a9c6-dd2edcc84514', embedding=None, metadata={'file_path': '../assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_name': '생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_type': 'application/pdf', 'file_size': 2596388, 'creation_date': '2024-04-25', 'last_modified_date': '2024-04-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='# 2023년도 중앙선거관리위원회 정책연구용역 보고서\n\n생성형 AI 신기술 도입에 따른 선거 규제 연구\n\n연구책임자: 김 (국립부경대학교) 주희\n\n공동연구자:\n\n차 (국립부경대학교) 재권\n\n김 (동아대학교) 현정\n\n조 (국민대학교) 성복\n\n연구보조원:\n\n박 (국립부경대학교) 서현\n\n권 (국립부경대학교) 수민\n\n국립부경대학교 산학협력단', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='70a

In [15]:
# 랭체인 도큐먼트로 변환
docs_langchain_format = [doc.to_langchain_format() for doc in documents]

idx = 0
print(f"page_content: {docs_langchain_format[idx].page_content}")
print(f"metadata: {docs_langchain_format[idx].metadata}")

page_content: # 2023년도 중앙선거관리위원회 정책연구용역 보고서

생성형 AI 신기술 도입에 따른 선거 규제 연구

연구책임자: 김 (국립부경대학교) 주희

공동연구자:

차 (국립부경대학교) 재권

김 (동아대학교) 현정

조 (국민대학교) 성복

연구보조원:

박 (국립부경대학교) 서현

권 (국립부경대학교) 수민

국립부경대학교 산학협력단
metadata: {'file_path': '../assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_name': '생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_type': 'application/pdf', 'file_size': 2596388, 'creation_date': '2024-04-25', 'last_modified_date': '2024-04-25'}


In [16]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 문서 분할
splits = text_splitter.split_documents(docs_langchain_format)

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

# 문서로부터 FAISS 인덱스 생성 및 검색기 설정
retriever = FAISS.from_documents(splits, embeddings).as_retriever(
    search_kwargs={"k": 10}
)

[Document(metadata={'file_path': '../assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_name': '생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_type': 'application/pdf', 'file_size': 2596388, 'creation_date': '2024-04-25', 'last_modified_date': '2024-04-25'}, page_content='# CISA는AI소프트웨어시스템과기술에대한교육을지속적으로실시할 것이며, AI전문성을갖춘인턴, 펠로우및미래직원을적극적으로채용할계획\n\n직원의AI전문성확대을위한교육으로AI기반소프트웨어시스템의기술적측면뿐만아니라법적·윤리적·정책적측면도반영\n\n출처: 한국지식재산연구원. 2023a. “미국 국토안보수사국, ‘AI 로드맵’ 발표.” (11월 24일) https://www.kiip.re.kr/board/trend/view.do?bd_gb=trend&bd_cd=1&bd_item=0&po_item_gb=US&po_no=22454(검색일:2023.11.24).\n\n미국은2023년11월14일,국토안보수사국(HSI)산하 사이버보안및인프라 보안국(CISA: Cybersecurity and Infrastructure Security Agency)은 인공지능의안전한개발및구현을보장하고자최초의‘AI 로드맵’을 발표하였다(<표4-1>참조).'),
 Document(metadata={'file_path': '../assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_name': '생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_type': 'application/pdf', 'file_size': 2596388, 'creation_date': '2024-04-25', 'last_modified_date': '2024-04-25'}, page_

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
docs = retriever.invoke("미국의 AI 로드맵은?")
docs

[Document(metadata={'file_path': '../assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_name': '생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_type': 'application/pdf', 'file_size': 2596388, 'creation_date': '2024-04-25', 'last_modified_date': '2024-04-25'}, page_content='# 주요국의 생성형 AI 선거 규제\n\n# 미국\n\n미국은 AI 산업을 선도하기 위해 AI 발전 전략 중심의 로드맵을 제시했다. AI 기술이 비약적으로 발전함에 따라, 미국 정부는 이에 관한 규제를 도입하고 있다. 미국 정부가 관련 영역에서 주요하게 다루는 바는 개인 권리 보호, 차별 금지, AI 기술을 이용한 안보 사항이다.'),
 Document(metadata={'file_path': '../assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_name': '생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_type': 'application/pdf', 'file_size': 2596388, 'creation_date': '2024-04-25', 'last_modified_date': '2024-04-25'}, page_content='# CISA는AI소프트웨어시스템과기술에대한교육을지속적으로실시할 것이며, AI전문성을갖춘인턴, 펠로우및미래직원을적극적으로채용할계획\n\n직원의AI전문성확대을위한교육으로AI기반소프트웨어시스템의기술적측면뿐만아니라법적·윤리적·정책적측면도반영\n\n출처: 한국지식재산연구원. 2023a. “미국 국토안보수사국, ‘AI 로드맵’ 발표.” (11월 24일) https://www.kiip.re.kr/board/trend/view.do?bd_gb=trend&bd_cd=1&bd_item=0&p

In [21]:
# 문서 출력 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )
    
# 결과 문서 출력
pretty_print_docs(docs)

Document 1:

# 주요국의 생성형 AI 선거 규제

# 미국

미국은 AI 산업을 선도하기 위해 AI 발전 전략 중심의 로드맵을 제시했다. AI 기술이 비약적으로 발전함에 따라, 미국 정부는 이에 관한 규제를 도입하고 있다. 미국 정부가 관련 영역에서 주요하게 다루는 바는 개인 권리 보호, 차별 금지, AI 기술을 이용한 안보 사항이다.
----------------------------------------------------------------------------------------------------
Document 2:

# CISA는AI소프트웨어시스템과기술에대한교육을지속적으로실시할 것이며, AI전문성을갖춘인턴, 펠로우및미래직원을적극적으로채용할계획

직원의AI전문성확대을위한교육으로AI기반소프트웨어시스템의기술적측면뿐만아니라법적·윤리적·정책적측면도반영

출처: 한국지식재산연구원. 2023a. “미국 국토안보수사국, ‘AI 로드맵’ 발표.” (11월 24일) https://www.kiip.re.kr/board/trend/view.do?bd_gb=trend&bd_cd=1&bd_item=0&po_item_gb=US&po_no=22454(검색일:2023.11.24).

미국은2023년11월14일,국토안보수사국(HSI)산하 사이버보안및인프라 보안국(CISA: Cybersecurity and Infrastructure Security Agency)은 인공지능의안전한개발및구현을보장하고자최초의‘AI 로드맵’을 발표하였다(<표4-1>참조).
----------------------------------------------------------------------------------------------------
Document 3:

AI위원회는 산업계, 학계, 공공부문에서 존경받는 각 분야의 리더들이 모여 AI 부문 협약의 핵심 약속을 이행하고 있다.

위원들은 분기별로 모여 AI 정책의 현재 우선순위, 기회, 과제에 대해 AI 사무국과 범정부적

In [22]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# 모델 초기화
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개의 문서 선택
compressor = CrossEncoderReranker(model=model, top_n=3)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# 압축된 문서 검색
compressed_docs = compression_retriever.invoke("미국의 AI 로드맵은?")

# 문서 출력
pretty_print_docs(compressed_docs)

Document 1:

# 주요국의 생성형 AI 선거 규제

# 미국

미국은 AI 산업을 선도하기 위해 AI 발전 전략 중심의 로드맵을 제시했다. AI 기술이 비약적으로 발전함에 따라, 미국 정부는 이에 관한 규제를 도입하고 있다. 미국 정부가 관련 영역에서 주요하게 다루는 바는 개인 권리 보호, 차별 금지, AI 기술을 이용한 안보 사항이다.
----------------------------------------------------------------------------------------------------
Document 2:

# 워싱턴

SB5152
제정
선거에영향을미치는합성미디어에대한신고

출처: NCSL. 2023.

# 소결

미국내 AI 관련 규제 초점

- 미국은 AI 산업을 선도하기 위해 AI 발전 전략 중심의 로드맵을 제시하였다.
- AI 기술이 비약적으로 발전함에 따라, 미국 정부는 이에 관한 규제를 도입하고 있다. 미국 정부가 관련 영역에서 주요하게 다루는 바는 개인 권리 보호, 차별 금지, AI 기술을 이용한 안보 사항이다.

미국내 선거, 정당에 관한 생성형 AI 창작물 유포 관련 논의
----------------------------------------------------------------------------------------------------
Document 3:

# 4. 주요국의 AI 및 생성형 AI 선도 규제

# 4.1 미국

가. AI 관련 입법 추진 개요

- AI로드맵
- 미국은 AI 산업 영역을 선도하는 기업을 보호하고, 발전을 육성하려는 의도와 자국의 안보를 강화하기 위한 의도에서 AI 및 생성형 AI 규제를 입법화하고 있다.
